# Physics Informed Neural Operators

In this notebook we will introduce the brief theory behind the Physics Informed Neural Operators (PINO) and use them to solve the same data-driven Darcy flow problem that was introduced in the [FNO notebook](Darcy_Flow_using_Fourier_Neural_Operators.ipynb)

### Learning Outcomes
1. How to setup and train PINO in Modulus
2. Defining a custom PDE constraint for grid data
3. Differences between PINO and FNO

## PINO Theory

The Physics-Informed Neural Operator (PINO) was introduced by Li et al. in the [paper](https://arxiv.org/abs/2111.03794). The PINO approach for surrogate modeling of PDE systems effectively combines the data-informed supervised learning framework of the FNO with the physics-informed learning framework. The PINO incorporates a PDE loss $\mathcal{L}_{pde}$ to the Fourier Neural Operator. This reduces the amount of data required to train a surrogate model, since the PDE loss constrains the solution space. 
The PDE loss also enforces physical constraints on the solution computed by a surrogate ML model, making it an attractive option as a verifiable, accurate and interpretable ML surrogate modeling tool.

We consider a stationary PDE system for simplicity, although the PINO method can be applied to dynamical systems as well. 
Following the notation used in the [paper](https://arxiv.org/abs/2111.03794), we consider a PDE represented by,

\begin{equation} 
\mathcal{P}(u, a) = 0 , \text{ in } D \subset \mathbb{R}^d,
\end{equation}

\begin{equation}
u = g ,  \text{ in } \partial D.
\end{equation}

Here, $\mathcal{P}$ is a Partial Differential Operator, $a$ are the coefficients/parameters and $u$ is the PDE solution.

In the FNO framework, the surrogate ML model is given by a the solution operator $\mathcal{G}^\dagger_{\theta}$, which maps any given coefficient in the coefficient space $a$ to the solution $u$. The FNO is trained in a supervised fashion using training data in the form of input-output pairs $\lbrace a_j, u_j \rbrace_{j = 1}^N$. The training loss for the FNO is given by summing the data loss, $\mathcal{L}_{data}(\mathcal{G}_\theta) = \lVert u - \mathcal{G}_\theta(a)  \rVert^2$ over all training pairs $\lbrace a_i, u_i,  \rbrace_{i=1}^N$,

In the PINO framework, the solution operator is optimized with an additional PDE loss given by $\mathcal{L}_{pde}(a, \mathcal{G}_{\theta}(a))$ computed over i.i.d. samples $a_j$ from an appropriate supported distribution in parameter/coefficient space.

In general, the PDE loss involves computing the PDE operator which in turn involves computing the partial derivatives of the Fourier Neural Operator ansatz. In general this is nontrivial. The key set of innovations in the PINO are the various ways to compute the partial derivatives of the operator ansatz. The PINO framework implements the differentiation in four different ways.

1. Numerical differentiation using a Finite-Difference Method (FDM).
2. Numerical differentiation computed via spectral derivative. 
3. Hybrid differentiation based on a combination of first-order "exact" derivatives and second-order FDM derivatives. 



## Problem Description

This problem illustrates developing a surrogate model that learns the mapping between a permeability and pressure field of
a Darcy flow system. The mapping learned, $\textbf{K} \rightarrow \textbf{U}$, should be true for a distribution of permeability fields $\textbf{K} \sim p(\textbf{K})$ and not just a single solution.

The key difference between PINO and FNO is that PINO adds a physics-informed term to the loss function of FNO. As discussed further in the theory, the PINO loss function is described by:

\begin{equation}
\mathcal{L} = \mathcal{L}_{data} + \mathcal{L}_{pde},
\end{equation}

where,
\begin{equation}
\mathcal{L}_{data} = \lVert u - \mathcal{G}_\theta(a)  \rVert^2 ,
\end{equation}

where $\mathcal{G}_\theta(a)$ is a FNO model with learnable parameters $\theta$ and input field $a$, and 
$\mathcal{L}_{pde}$ is an appropriate PDE loss. For the 2D Darcy problem this is given by

\begin{equation}
\mathcal{L}_{pde} = \lVert -\nabla \cdot \left(k(\textbf{x})\nabla \mathcal{G}_\theta(a)(\textbf{x})\right) - f(\textbf{x}) \rVert^2 ,
\end{equation}

where $k(\textbf{x})$ is the permeability field, $f(\textbf{x})$ is the forcing function equal to 1 in this case, and $a=k$ in this case.

Note that the PDE loss involves computing various partial derivatives of the FNO ansatz, $\mathcal{G}_\theta(a)$. 
In general this is nontrivial; in Modulus, three different methods for computing these are provided. These are based on the original PINO paper:

1. Numerical differentiation computed via Finite-Difference Method (FDM)
2. Numerical differentiation computed via spectral derivative
3. Hybrid differentiation based on a combination of first-order "exact" derivatives and second-order FDM derivatives

The first 2 approaches are the same as proposed in the original paper. The third approach is a modification of the "exact" approach proposed in the paper.
This method is slower and more memory intensive than the numerical derivative approaches when computing second order derivatives
because it requires the computation of a Hessian matrix. 
Instead, a "hybrid" approach is provided which offers a compromise by combining first-order "exact"(the exact method is not technically exact because it uses a combination of numerical spectral derivatives and exact differentiation, see original paper for more details) derivatives and second-order FDM derivatives.




## Case setup

The setup for this problem is largely the same as the FNO example, except that the PDE loss is defined and the FNO model is constrained using it. This process is described in detail when we define the PDE loss. 

Similar to the FNO notebook, the training and validation data for this example can be found on the [Fourier Neural Operator Github page](https://github.com/zongyi-li/fourier_neural_operator). The data can be downloaded using an automated script similar to the FNO notebook. 

**Note:** In this notebook we will walk through the contents of [`darcy_PINO.py`](../../source_code/darcy/darcy_PINO.py) script. 

## Configuration

The configuration for this problem is similar to the FNO example, but importantly there is an extra parameter `custom.gradient_method` where the method for computing the gradients in the PDE loss is selected. This can be one of `fdm`, `fourier`, `hybrid` corresponding to the three options above. The balance between the data and PDE terms in the loss function can also be controlled using the `loss.weights` parameter group. The contents of the [`config_PINO.yaml`](../../source_code/darcy/conf/config_PINO.yaml) are shown below. 

```yaml
defaults :
  - modulus_default
  - /arch/conv_fully_connected_cfg@arch.decoder
  - /arch/fno_cfg@arch.fno
  - scheduler: tf_exponential_lr
  - optimizer: adam
  - loss: sum
  - _self_

cuda_graphs: false
jit: false

custom:
  gradient_method: hybrid
  ntrain: 1000
  ntest: 100

arch:
  decoder:
    input_keys: [z, 32]
    output_keys: sol
    nr_layers: 1
    layer_size: 32

  fno:
    input_keys: coeff
    dimension: 2
    nr_fno_layers: 4
    fno_modes: 12
    padding: 9

scheduler:
  decay_rate: 0.95
  decay_steps: 1000

training:
  rec_results_freq : 1000
  max_steps : 10000

loss:
  weights:
    sol: 1.0
    darcy: 0.1

batch_size:
  grid: 8
  validation: 8
```

## Define PDE Loss for grid data

For this example, a custom PDE residual calculation is defined using the various approaches proposed above. The process of defining a custom PDE residual using sympy and auto-diff was discussed in the [notebooks on PINNs](../diffusion_1d/Diffusion_Problem_Notebook.ipynb). For this problem, we will not be relying on standard auto-diff for calculating the derivatives, instead we want to explicitly define how the residual is calculated using a custom `torch.nn.Module` called `Darcy`. The purpose of this module is to compute and return the Darcy PDE residual given the input and output tensors of the FNO model, which is done via its `.forward(...)` method 

```python
# helper function for computing spectral derivatives
from modulus.models.layers.spectral_layers import fourier_derivatives 
# helper function for computing finite difference derivatives
from ops import dx, ddx 
```

```python
class Darcy(torch.nn.Module):
    "Custom Darcy PDE definition for PINO"

    def __init__(self, gradient_method: str = "hybrid"):
        super().__init__()
        self.gradient_method = str(gradient_method)

    def forward(self, input_var: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        # get inputs
        u = input_var["sol"]
        c = input_var["coeff"]
        dcdx = input_var["Kcoeff_y"]  # data is reversed
        dcdy = input_var["Kcoeff_x"]

        dxf = 1.0 / u.shape[-2]
        dyf = 1.0 / u.shape[-1]
        # Compute gradients based on method
        # Exact first order and FDM second order
        if self.gradient_method == "hybrid":
            dudx_exact = input_var["sol__x"]
            dudy_exact = input_var["sol__y"]
            dduddx_fdm = input_var["sol__x__x"]
            dduddy_fdm = input_var["sol__y__y"]
            # compute darcy equation
            darcy = (
                1.0
                + (dcdx * dudx_exact)
                + (c * dduddx_fdm)
                + (dcdy * dudy_exact)
                + (c * dduddy_fdm)
            )
        # FDM gradients
        elif self.gradient_method == "fdm":
            dudx_fdm = dx(u, dx=dxf, channel=0, dim=0, order=1, padding="replication")
            dudy_fdm = dx(u, dx=dyf, channel=0, dim=1, order=1, padding="replication")
            dduddx_fdm = ddx(
                u, dx=dxf, channel=0, dim=0, order=1, padding="replication"
            )
            dduddy_fdm = ddx(
                u, dx=dyf, channel=0, dim=1, order=1, padding="replication"
            )
            # compute darcy equation
            darcy = (
                1.0
                + (dcdx * dudx_fdm)
                + (c * dduddx_fdm)
                + (dcdy * dudy_fdm)
                + (c * dduddy_fdm)
            )
        # Fourier derivative
        elif self.gradient_method == "fourier":
            dim_u_x = u.shape[2]
            dim_u_y = u.shape[3]
            u = F.pad(
                u, (0, dim_u_y - 1, 0, dim_u_x - 1), mode="reflect"
            )  # Constant seems to give best results
            f_du, f_ddu = fourier_derivatives(u, [2.0, 2.0])
            dudx_fourier = f_du[:, 0:1, :dim_u_x, :dim_u_y]
            dudy_fourier = f_du[:, 1:2, :dim_u_x, :dim_u_y]
            dduddx_fourier = f_ddu[:, 0:1, :dim_u_x, :dim_u_y]
            dduddy_fourier = f_ddu[:, 1:2, :dim_u_x, :dim_u_y]
            # compute darcy equation
            darcy = (
                1.0
                + (dcdx * dudx_fourier)
                + (c * dduddx_fourier)
                + (dcdy * dudy_fourier)
                + (c * dduddy_fourier)
            )
        else:
            raise ValueError(f"Derivative method {self.gradient_method} not supported.")

        # Zero outer boundary
        darcy = F.pad(darcy[:, :, 2:-2, 2:-2], [2, 2, 2, 2], "constant", 0)
        # Return darcy
        output_var = {
            "darcy": dxf * darcy,
        }  # weight boundary loss higher
        return output_var
```

The gradients of the FNO solution are computed according to the gradient method selected above. The FNO model automatically outputs first order gradients when the `hybrid` method is used, and so no extra computation of these is necessary. Furthermore, note that the gradients of the permeability field are already included as tensors in the FNO input training data (with keys `Kcoeff_x` and `Kcoeff_y`) and so these do not need to be computed.

Next, incorporate this module into Modulus by wrapping it into a Modulus `Node`. This ensures the module is incorporated into Modulus’ computational graph and can be used to optimise the FNO.

```python
from modulus.node import Node
```

```python
    # Make custom Darcy residual node for PINO
    inputs = [
        "sol",
        "coeff",
        "Kcoeff_x",
        "Kcoeff_y",
    ]
    if cfg.custom.gradient_method == "hybrid":
        inputs += [
            "sol__x",
            "sol__y",
        ]
    darcy_node = Node(
        inputs=inputs,
        outputs=["darcy"],
        evaluate=Darcy(gradient_method=cfg.custom.gradient_method),
        name="Darcy Node",
    )
```

## Loading the Data and Initializing the Model

These sections follow similar processes as the FNO example. Only for the case where `hybrid` gradient method is used, you need to additionally instruct the model to output the appropriate gradients by specifying these gradients in its output keys. 

```python
    # load training/ test data
    input_keys = [
        Key("coeff", scale=(7.48360e00, 4.49996e00)),
        Key("Kcoeff_x"),
        Key("Kcoeff_y"),
    ]
    output_keys = [
        Key("sol", scale=(5.74634e-03, 3.88433e-03)),
    ]

    download_FNO_dataset("Darcy_241", outdir="datasets/")
    invar_train, outvar_train = load_FNO_dataset(
        "datasets/Darcy_241/piececonst_r241_N1024_smooth1.hdf5",
        [k.name for k in input_keys],
        [k.name for k in output_keys],
        n_examples=cfg.custom.ntrain,
    )
    invar_test, outvar_test = load_FNO_dataset(
        "datasets/Darcy_241/piececonst_r241_N1024_smooth2.hdf5",
        [k.name for k in input_keys],
        [k.name for k in output_keys],
        n_examples=cfg.custom.ntest,
    )

    # add additional constraining values for darcy variable
    outvar_train["darcy"] = np.zeros_like(outvar_train["sol"])

    train_dataset = DictGridDataset(invar_train, outvar_train)
    test_dataset = DictGridDataset(invar_test, outvar_test)

    # Define FNO model
    decoder_net = instantiate_arch(
        cfg=cfg.arch.decoder,
        output_keys=output_keys,
    )
    fno = instantiate_arch(
        cfg=cfg.arch.fno,
        input_keys=[input_keys[0]],
        decoder_net=decoder_net,
    )
    if cfg.custom.gradient_method == "hybrid":
        derivatives = [
            Key("sol", derivatives=[Key("x")]),
            Key("sol", derivatives=[Key("y")]),
            Key("sol", derivatives=[Key("x"), Key("x")]),
            Key("sol", derivatives=[Key("y"), Key("y")]),
        ]
        fno.add_pino_gradients(
            derivatives=derivatives,
            domain_length=[1.0, 1.0],
        )

    # Make custom Darcy residual node for PINO
    inputs = [
        "sol",
        "coeff",
        "Kcoeff_x",
        "Kcoeff_y",
    ]
    if cfg.custom.gradient_method == "hybrid":
        inputs += [
            "sol__x",
            "sol__y",
        ]
    darcy_node = Node(
        inputs=inputs,
        outputs=["darcy"],
        evaluate=Darcy(gradient_method=cfg.custom.gradient_method),
        name="Darcy Node",
    )
    nodes = [fno.make_node('fno'), darcy_node]
```

## Adding Constraints

Finally, add constraints to your model in a similar fashion to the FNO example.

```python
    # make domain
    domain = Domain()

    # add constraints to domain
    supervised = SupervisedGridConstraint(
        nodes=nodes,
        dataset=train_dataset,
        batch_size=cfg.batch_size.grid,
    )
    domain.add_constraint(supervised, "supervised")
```

The process of defining the validator and using the `Solver` class remains the same as the FNO example. 

## Training the Model 

The training can now be simply started by executing the python script

In [ ]:
import os
os.environ["RANK"]="0"
os.environ["WORLD_SIZE"]="1"
os.environ["MASTER_ADDR"]="localhost"

In [ ]:
!python ../../source_code/darcy/darcy_PINO.py

## Results and Post-processing

The network directory folder contains several plots of different validation predictions. One of them is shown below.

PINO validation predictions. (Left to right) Input permeability and its spatial derivatives, true pressure, predicted pressure, error.

<img src="pino_darcy_pred.png" alt="Drawing" style="width: 900px;"/>

### Comparison to FNO

The Tensorboard plot below compares the validation loss of PINO (all three gradient methods) and FNO. You can see that with large amounts of training data (1000 training examples), both FNO and PINO perform similarly. 

<img src="pino_darcy_tensorboard1.png" alt="Drawing" style="width: 600px;"/>

A benefit of PINO is that the PDE loss regularizes the model, meaning that it can be more efficient in "small data" regimes. The plot below shows the validation loss when both models are trained with only 100 training examples. In this case, we find that PINO outperforms FNO. 

<img src="pino_darcy_tensorboard2.png" alt="Drawing" style="width: 600px;"/>

# Licensing
This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).
